### Dynamically returning directly 
In this example we will buiild a chat execture wher ethe LLM can optionally decide to return the result of a tool call as the final answer (isn't this what we have been doing all along?). This is useful for cases hwere you have a tools that can sometimes generate responses that are acceptable as final answers and you want to use the LLM to determine when this is the case. 


##### Setup tools 
The first step in an agent based application is to setup the tools required for the agent


In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.pydantic_v1 import BaseModel, Field

In [3]:
# defining the tool arg schema
class SearchTool(BaseModel):
    """ Look up things online, optionally returning directly"""
    query: str = Field(description="query to look up online")
    return_directly: bool = Field(description="Whether or not the result of this should be returned directly to the user without you seeing what it is", default=False)

In [4]:
tools = [TavilySearchResults(max_results=2, args_schema=SearchTool)]

In [5]:
from langchain.tools.render import format_tool_to_openai_function
functions = [format_tool_to_openai_function(tool) for tool in tools]

In [6]:
from langgraph.prebuilt import ToolExecutor
tool_executor = ToolExecutor(tools)

##### Setup the model
We will be using the Mixtral 8x7b model as our language model and bind the functions to the model

In [7]:
from langchain_openai.chat_models import ChatOpenAI
model = ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1", temperature=0)

In [8]:
model = model.bind_functions(functions=functions)

##### Define the state of the graph
We would be making using of the Stateful Graph which is the most common type of the langgraph graph

In [9]:
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage
import operator

In [10]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

##### Define the nodes within the graph
Now we have defined the state dictionary of our chain we can now define the nodes we would like to use within our graph

In [11]:
from langgraph.prebuilt import ToolInvocation
import json 
from langchain_core.messages import FunctionMessage

In [12]:
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message: BaseMessage = messages[-1]
    if "tool_calls" not in last_message.additional_kwargs:
        return "end"
    else:
        arguments = json.loads(last_message.additional_kwargs["tool_calls"][0]["function"]["arguments"])
        if arguments.get("return_directly", False):
            return "final"
        else:
            return "continue"
def call_function(state: AgentState):
    messages = state["messages"]
    last_message: BaseMessage = messages[-1]
    
    fn = last_message.additional_kwargs["tool_calls"][0]["function"]
    tool_name = fn["name"]
    arguements = json.loads(fn["arguments"])
    
    
    if tool_name == "tavily_search_results_json":
        if "return_directly" in arguements:
            del arguements["return_directly"]
            
    action = ToolInvocation(
        tool=tool_name, 
        tool_input=arguements
    )
    
    response = tool_executor.invoke(action)
    
    return {"messages": [FunctionMessage(content=str(response), name=action.tool)]}

def call_model(state: AgentState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

##### Define the graph and compile it into an app
Now we have defined the nodes and conditions within our graph we can then put all of that together and define our entire graph architecture. 

In [13]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

In [14]:
workflow.add_node("agent", call_model)
workflow.add_node("action", call_function)
workflow.add_node("final", call_function)

workflow.set_entry_point("agent")

In [15]:
workflow.add_conditional_edges("agent", should_continue, {"final": "final", "continue": "action", "end": END})
workflow.add_edge("action", "agent")
workflow.add_edge("final", END)

In [16]:
app = workflow.compile()

##### Run our langgraph by invoking it like a runnable
The langraph app uses the same runnable interface as chain, this allows us to use it the same way we define our chains in langchain

In [ ]:
def steam_graph(inputs, app):
    for output in app.stream(inputs):
        for key, value in output.items():
            print(f"Output from node {key}:")
            print("---")
            print(value)
        print("---")
        

In [17]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xjeythg7fexfmy6gbsopsek8', 'function': {'arguments': '{"query":"weather in sf","return_directly":false}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]})]}

---

Output from node 'action':
---
{'messages': [FunctionMessage(content='[{\'url\': \'https://weather.com/weather/hourbyhour/l/USCA0987:1:US\', \'content\': "recents\\nSpecialty Forecasts\\nHourly Weather-San Francisco, CA\\nBeach Hazard Statement\\nSaturday, November 25\\n5 pm\\nClear\\n6 pm\\nClear\\n7 pm\\nClear\\n8 pm\\nMostly Clear\\n9 pm\\nPartly Cloudy\\n10 pm\\nPartly Cloudy\\n11 pm\\nPartly Cloudy\\nSunday, November 26\\n12 am\\nPartly Cloudy\\n1 am\\nMostly Cloudy\\n2 am\\nMostly Cloudy\\n3 am\\nMostly Cloudy\\n4 am\\nCloudy\\n5 am\\nCloudy\\n6 am\\nMostly Cloudy\\n7 am\\nMostly Cloudy\\n8 am\\nMostly Cloudy\\n9 am\\nMostly Cloudy\\n10 am\\nPartly Cloudy\\n11 am\\nPartly Cloudy\\n12 pm\\nPartly 